# Advanced grid-search demo
Download the Kaggle data, run grid-search training, evaluate fairness and calibrate the models.

In [ ]:
from pathlib import Path
from scripts import download_data
from src.models import logreg, cart
download_data.main()
df = logreg.load_data()
gs_lr = logreg.grid_train_from_df(df, artefact_path=Path('artefacts/logreg_gs.joblib'))
gs_cart = cart.grid_train_from_df(df, artefact_path=Path('artefacts/cart_gs.joblib'))


In [ ]:
from src.evaluate import evaluate_models
from src.evaluation_utils import youden_thr, four_fifths
metrics = evaluate_models(df, group_col='Gender')
thr = youden_thr(gs_lr.best_estimator_, df.drop(columns=[logreg.TARGET]), df[logreg.TARGET])
ff = four_fifths(gs_lr.best_estimator_, df.drop(columns=[logreg.TARGET]), df[logreg.TARGET], 'Gender', thr)
metrics, ff


In [ ]:
from src.calibration import calibrate_model
import joblib
X = df.drop(columns=[logreg.TARGET])
y = df[logreg.TARGET]
cal_lr = calibrate_model(gs_lr.best_estimator_, X, y)
cal_cart = calibrate_model(gs_cart.best_estimator_, X, y)
joblib.dump(cal_lr, 'artefacts/logreg_calibrated.joblib')
joblib.dump(cal_cart, 'artefacts/cart_calibrated.joblib')
